# Aplicação de Hypertuning aos modelos de machine learning 
* Este código de previsão Multi-step aplica diferentes métodos de hypertuning aos modelos MLP, LSTM, Árvore de Decisão e Random Forest.

# Importação de bibliotecas 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import tensorflow as tf
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Conv1D, MaxPooling1D,  Bidirectional, Flatten 
from sklearn.model_selection import cross_val_score
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
import time
from sklearn.model_selection import RandomizedSearchCV, KFold
import os
import warnings
warnings.filterwarnings("ignore")

# Lista de arquivos que estão na pasta indicada como raiz 
* O comando os.listdir retorna uma lista com o nome de todos os arquivos contidos na pasta

In [ ]:
file = r'C:\Users\eluar\Documents\Estudo_IA_Python\Artigo_Previ_Dados_MET\BASE_DADOS_MET'
arquivos = os.listdir(file)

# Montar janelas para previsão Multi-Step
* Esta função recebe um conjunto de dados e monta as janelas de previsão juntamente com os steps a serem previstos 

In [ ]:
def montarJanela(data, janela, step):
    
    x, y = [],[]
    
    for i in range(janela, len(data)):
        
        fim_y = i+step
        if fim_y > len(data):
            break
        
        x.append(data[i-janela:i,0])
        y.append(data[i:fim_y,0])
        
    return np.array(x), np.array(y)

# Cria as variáveis que definem o número de entradas e saídas das RNAs 
* TM_janela é a variável utilizada para determinar o número de entradas das RNAS (Janela de previsão)
* TM_step é a variável utilizada para determinar o número de saídas das RNAS (Passos a frete)

In [ ]:
TM_janela = 0
TM_step = 0

# Função para criar uma RNA CNN-LSTM

In [ ]:
def criarRedeLSTM(optimizer, activation1, activation2, neurons1, neurons2, neurons3):
    Regressor = Sequential()
    Regressor.add(Conv1D(filters = 128, kernel_size = 2, activation = activation1, input_shape=(TM_janela, 1)))
    Regressor.add(MaxPooling1D(pool_size = 1))
    Regressor.add(LSTM(units = neurons1, return_sequences = True, activation = 'tanh'))
    Regressor.add(Dropout(0.2))
    Regressor.add(LSTM(units = neurons2, activation = 'tanh'))
    Regressor.add(Dropout(0.2))
    Regressor.add(Dense(units = neurons3, activation = activation2))
    Regressor.add(Dropout(0.1))
    Regressor.add(Dense(TM_step, activation = 'linear'))
    
    Regressor.compile(loss= 'mean_squared_error', optimizer= optimizer, metrics=['mean_absolute_error'])
    return Regressor


# Modelos Random Search

In [ ]:
def criarLSTMRD(x_train, y_train, janela, step, local_arquivo):
    
    #Altera a entrada para um formato compativél com a as entradas da RNA LSTM 
    #x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
    
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))   
    
    print(np.shape(x_train))
    print(np.shape(y_train))
    
    start = time.time() #Tempo inicial
    SEED = 10
    np.random.seed(SEED)
    
    regressor = KerasRegressor(build_fn= criarRedeLSTM)
    
    parameters = {'batch_size':[128, 256, 512],
                  'optimizer':['adam', 'sgd', 'RMSprop'],
                  'activation1':['linear','relu','elu','tanh', 'sigmoid'],
                  'activation2':['linear','relu','elu','tanh', 'sigmoid'],
                  'neurons1':[400, 200],
                  'neurons2':[100, 50], 
                  'neurons3':[50, 25]
                 }
    buscaLSTM = RandomizedSearchCV(regressor, 
                                   parameters,
                                   n_iter = 10,
                                   scoring='r2',
                                   return_train_score=True,
                                   cv = KFold(n_splits = 5, shuffle = True),
                                   random_state = SEED
                                  )
    
    buscaLSTM = buscaLSTM.fit(x_train, y_train, epochs = 400, verbose = 0)
    end = time.time() #Tempo final 
    tempo = round(end-start, 4) #calcula o tempo de execução do modelo
   
    # Pega os melhores parametros 
    melhores_parametros = buscaLSTM.best_params_
    melhor_precisao = buscaLSTM.best_score_ # Verificar Scores dos modelos 

    print("----------//------Resultados LSTM RD--------//-----------")
    print("O tempo de execução LSTM: ", tempo)
    print(melhores_parametros)
    print(melhor_precisao)
    print("----------//-----------------------------------------//-----------")
    
    #Criar o arquivo de texto no diretório para salvar as informações de configuração do modelo
    titulo = "----------//------Resultados LSTM Random Search--------//-----------"
    final = "----------//-------------------------------------------//------------"
    with open('LSTM_RD_SEARCH.txt', 'a') as arquivo:
        arquivo.write(f'{titulo}\nLocalidade_municipio: {local_arquivo}\nJanela: {janela}\nStep: {step}\nTempo:{tempo}\nMelhores Parametros: {melhores_parametros}\nPrecisao r2: {melhor_precisao}\n{final}\n')

        
    
def criarRDForestRD(x_train, y_train, janela, step, local_arquivo): 
    
    start = time.time() #Tempo inicial
    SEED = 10
    np.random.seed(SEED)

    """
       Esses parâmetros servem para restringir a liberdade da árvore a fim de evitar o sobreajuste
       max_depth =  profundidade maxima da árvore
       min_samples_split = número mínimo de amostras que um nó deve ter antes que possa ser dividido
       min_samples_leaf = número mínimo de amostras que um nó da folha deve ter 
       max_leaf_nodes = número máximo de nós da folha 
    """
    parameters = {'n_estimators':[10, 20, 30, 40, 50, 60, 70, 80],
                  'max_depth':[3, 6, 9, 12, 15, 18, 21, 24],
                  'min_samples_split':[16, 32, 64, 128, 256, 512],
                  'min_samples_leaf':[16, 32, 64, 128, 256, 512], 
                  'max_leaf_nodes':[12, 24, 36, 48, 60, 72, 84, 96]
                 }
    
    buscaRF = RandomizedSearchCV(RandomForestRegressor(), 
                                 parameters,
                                 n_iter = 200,
                                 scoring='r2',
                                 return_train_score=True,
                                 cv = KFold(n_splits = 5, shuffle = True),
                                 random_state = SEED
                                )
    
    buscaRF = buscaRF.fit(x_train, y_train)
    end = time.time() #Tempo final 
    tempo = round(end-start, 4) #calcula o tempo de execução do modelo
   
    # Pega os melhores parametros 
    melhores_parametros = buscaRF.best_params_
    melhor_precisao = buscaRF.best_score_

    print("----------//------Resultados Random Forest RD--------//-----------")
    print("O tempo de execução random forest e: ", tempo)
    print(melhores_parametros)
    print(melhor_precisao)
    print("----------//-----------------------------------------//-----------")
    
    #Criar o arquivo de texto no diretório para salvar as informações de configuração do modelo
    titulo = "----------//------Resultados Random Forest Random Search--------//-----------"
    final = "----------//----------------------------------------------------//------------"
    with open('RD_FOREST_RD_SEARCH.txt', 'a') as arquivo:
        arquivo.write(f'{titulo}\nLocalidade_municipio: {local_arquivo}\nJanela: {janela}\nStep: {step}\nTempo:{tempo}\nMelhores Parametros: {melhores_parametros}\nPrecisao r2: {melhor_precisao}\n{final}\n')
    

def criarArvoreRD(x_train, y_train, janela, step, local_arquivo):
    
    start = time.time()# Tempo inicial
    SEED = 10
    np.random.seed(SEED)
    
    """
       Esses parâmetros servem para restringir a liberdade da árvore a fim de evitar o sobreajuste
       max_depth =  profundidade maxima da árvore
       min_samples_split = número mínimo de amostras que um nó deve ter antes que possa ser dividido
       min_samples_leaf = número mínimo de amostras que um nó da folha deve ter 
       max_leaf_nodes = número máximo de nós da folha 
    """
    parameters = {'max_depth':[3, 6, 9, 12, 15, 18, 21, 24],
                  'min_samples_split':[32, 64, 128, 256, 512],
                  'min_samples_leaf':[32, 64, 128, 256, 512], 
                  'max_leaf_nodes':[12, 24, 36, 48, 60, 72, 84, 96]
                 }
    
    buscaRD = RandomizedSearchCV(DecisionTreeRegressor(), 
                                 parameters,
                                 n_iter = 200,
                                 scoring='r2',
                                 return_train_score=True,
                                 cv = KFold(n_splits = 5, shuffle = True),
                                 random_state = SEED
                                )
    
    buscaRD = buscaRD.fit(x_train, y_train)
    end = time.time() #Tempo final 
    tempo = round(end-start, 4) #calcula o tempo de execução do modelo
   
    # Pega os melhores parametros 
    melhores_parametros = buscaRD.best_params_
    melhor_precisao = buscaRD.best_score_

    print("----------//------Resultados Arvore de decisão RD--------//-----------")
    print("O tempo de execução da Arvore e: ", tempo)
    print(melhores_parametros)
    print(melhor_precisao)
    print("----------//-----------------------------------------//-----------")
    
    #Criar o arquivo de texto no diretório para salvar as informações de configuração do modelo
    titulo = "----------//------Resultados Arvore Decisao Random Search--------//-----------"
    final = "----------//-----------------------------------------------------//------------"
    with open('ARVORE_DECISAO_RD_SEARCH.txt', 'a') as arquivo:
        arquivo.write(f'{titulo}\nLocalidade_municipio: {local_arquivo}\nJanela: {janela}\nStep: {step}\nTempo:{tempo}\nMelhores Parametros: {melhores_parametros}\nPrecisao r2: {melhor_precisao}\n{final}\n')



# Abre os arquivos, normaliza e aplica os modelos  com dierentes janelas de previsão 

In [ ]:
cont = 0 
while (cont < len(arquivos)):
    
    #Monta o caminho com um arquivo específico 
    caminho = r""+file+"\\"+arquivos[cont]
    
    #Abre o arquivo na pasta 
    df = pd.read_excel(caminho)
    #display(df)
    
    #atribui os dados a variável DTF
    DTF = df[['PRECIPITACAO(mm) (ACM)']]
    #VENTO_VEL_H (m/s) (MD)  PRECIPITACAO(mm) (ACM) UMID_REL_AR_H (%) (MD) TEMPMED (C) (MD)
    
    # Muda a escala dos valores para 0 a 1
    scaler = MinMaxScaler()
    dataF = scaler.fit_transform(DTF)
    
    
    #define o tamanho das janelas e o número de passos a frente para previsão 
    janelas = [3]
    steps = [1, 2, 3, 4]
    
    for i in janelas:
        for j in steps:
            x, y = montarJanela(dataF, i, j)
            print(f"Tamanho Janela x QTD Step: {i} x {j} ")
            
            #Parametros necessários para execução da MLP TM_janela, TM_step
            global TM_janela, TM_step
            TM_janela = i 
            TM_step = j
            #---------------------------//--------------//---------------------
            #criarLSTMRD(x, y, i, j, arquivos[cont])
            criarRDForestRD(np.array(x), np.array(y), i, j, arquivos[cont])
            #criarArvoreRD(x, y, i, j, arquivos[cont])
    cont = cont+1